# HTTP 靜態網頁架構說明

* 認識靜態網頁的溝通架構與運作原理
* HTML、CSS、JavaScript 在網頁中扮演的角色
* 網頁中定位資料的方法

## 作業目標

1. （簡答題）請問目前提到的這種基於 HTTP 一來一回的網頁傳輸方式，有哪些缺點及問題？
2. （簡答題）網頁中使用的 JavaScipt 程式語言，跟我們一般使用的 Python 在執行上有什麼不同。


### 1.（簡答題）請問目前提到的這種基於 HTTP 一來一回的網頁傳輸方式，有哪些缺點及問題？

https://www.ithome.com.tw/voice/94371

HTTP 底層的協定是TCP，因此，當HTTP的客戶端想要取得一個檔案資源時，就必須在一個TCP連線上發出請求。HTTP是一個基於「請求－回應」的協定，也是說，總是由客戶端發出請求，而伺服器端對應一個回應。

在HTTP的伺服器端收到請求資訊後，會開始處理該請求，在完成請求的處理之後，開始回傳回應的內容。當HTTP伺服器端在處理請求時，整個TCP連線其實處於一個閒置的情況，此外，客戶端所能做的事也只有等待。

而且，通常一個要能夠在瀏覽器中瀏覽完整的網頁內容，這中間涉及許多的檔案需要透過HTTP去取得，而單一個TCP連線只能同時間處理一個檔案，為此，瀏覽器通常都會同時建立多個連線，以利更快的取得多個檔案的內容。否則，以HTTP的天性，必須逐一等待伺服器傳輸完前一個檔案後，才能夠再繼續取得下一個檔案。

面對傳輸效率不佳的狀況

在最早的HTTP/1.0協定中，每次發出一個HTTP的請求都需要重新建立一個TCP連線，當該請求的回應內容傳輸完畢之後，該TCP連線即會被切斷，而這是一個非常沒有效率的事情，怎麼說呢？

第一個原因，是TCP在建立連線時，連線的兩方需要完成一個所謂3-Way Handshaking的動作，這會造成不小的延遲。對於一些每建立一個TCP連線，就會持續使用很長一段時間的應用來說，這個初始建立連線的延遲一點也不重要。例如，透過telnet協定連往BBS站時，只會建立一個TCP連線，卻可以使用很長一段時間，這段建立連線所造成的延遲，就不影響整個大局。

但是，對基於「請求－回應」模式的HTTP來說，如果透過HTTP回傳的檔案不夠大時，例如一個只有幾十 KB的HTML檔案，它可能不需要太多時間就可以完成傳輸，那麼花在建立TCP連線上的延遲，佔整體的比例就會高出很多。

在HTTP/1.0中更糟的是，一旦完成傳輸後，就會切斷TCP連線，之後倘若要請求另一個檔案，又必須重新建立一個全新的TCP連線，這使得每次都需要反覆不斷花費高昂的代價，在建立 TCP 連線之上，但每個TCP連線，卻又可能只傳輸少許的資料，就又被切斷了。

為此，在HTTP/1.1中，增加了讓連線「保持存活（Keep Alive）」的標頭（header），讓客戶端及伺服器端可以協調重複運用同一個TCP連線，每當客戶端接收完來自伺服器端的回應內容後，可以繼續在同一 TCP 連線裡發出下一個請求。

但這樣的設計可以讓情況好轉，但並沒有辦法完全解決，因為這個「保持存活」的情況，若是客戶端在一段時間內，並沒有繼續在TCP連線中發出下一個請求，此TCP連線亦會被切斷。

讓我們想想網頁瀏覽的行為模式，通常都是在載入完諸多檔案完畢後，使用者開始花時間瀏覽。在載入諸多檔案時，「保持存活」的特性，可以避免重新建立太多TCP連線，但是當使用者在瀏覽網頁時，瀏覽器常不會再發送太多的請求至伺服器端，此時，先前已建立的TCP連線就會被切斷。等待使用者再連結到下一個網頁時，瀏覽器仍然必須重新建立若干個全新的TCP連線。

建立TCP連線的額外負擔當中，除了上述的3-Way Handshaking之外，還有一個部分，就是 TCP 的「緩起步（ slow start）」特性。

TCP本身是一個具有流量控制（flow control），以及擁塞控制（congestion control）能力的協定，因此，它會試著估算單位時間內要傳輸多少資料量最有效率。當頻寬本身不足時，若是單位時間內試著傳輸太多的資料量至另一端，但卻無法即時傳輸完成，就會造成擁塞。另一方面，若是頻寬充足，但卻傳輸的太少，又會造成效率不彰、無法善用頻寬的情況。

因此，TCP的演算法會盡量優化此事，而緩起步正是其演算法中的一環。TCP會逐步視情況擴展單位時間內所傳輸的資料量，但在網路連線剛建立之際，它會試著從很小的傳輸量開始嘗試，這使得在連線剛建立的初期，無法善用實際上可能十分充足的頻寬。

會產生很多短命的TCP連線

就和 3-Way Handshaking 一樣，對於那種生命期很短的TCP連線來說，所造成的延遲影響比例就相對高出許多。但HTTP協定本身，就傾向於製造出諸多生命期很短的TCP連線，因此，我們可以說，因為 HTTP 的天性，使得這些延遲產生出比較大的負面效應。

此外，同時間多個TCP連線並行傳輸的情況，也可能讓 TCP 演算法在做流量及擁塞控制時的估算失準，造成了無法在 TCP 之上進行高效傳輸的結果。而每個客戶端都會同時和伺服器端建立多個 TCP 連線的行為，也使得伺服器必須配置更多的網路連線資源來處理，例如佔用更多的sockets及作業系統中的資源。而為了處理更多的連線請求，在多執行緒或程序的資源負擔，也變得更重。

所以總的來看，同時間多連線及短生命期傾向的TCP連線，正是HTTP在效率上打折扣的原因。而這樣的觀察，也正一步一步的導引著、觸發著 HTTP改版的契機，其中影響最深遠的，莫過於 Google 的SPDY了。而有了SPDY協定，才催生了之後改版的HTTP/2。

### 2. （簡答題）網頁中使用的 JavaScipt 程式語言，跟我們一般使用的 Python 在執行上有什麼不同。

https://codertw.com/%E7%A8%8B%E5%BC%8F%E8%AA%9E%E8%A8%80/15815/

